In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights
!pip install filterpy==1.4.5
from object_detection import *
from object_tracking import *
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
class_threshold = 0.5
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
	"boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
	"bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
	"backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
	"sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
	"tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
	"apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
	"chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
	"remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
	"book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]
object_detector = make_model()
weight_reader = WeightReader('yolov3.weights')
weight_reader.load_weights(object_detector)
object_tracker = ObjectTracker(max_age=1,min_hits=3,iou_threshold=0.3)
save_tracks=[]
cap= cv2.VideoCapture('/content/drive/MyDrive/video/cars.mp4')
j=1
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == False:
    break
  print(j)
  j += 1
  input_w, input_h = 416, 416
  image_h,image_w,_=frame.shape
  image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  image=img_to_array(cv2.resize(image,(416,416)))
  image=image.astype('float32')
  image /= 255.0
  image=expand_dims(image,0) 
  yhat = object_detector.predict(image)
  boxes = list() 
  for i in range(len(yhat)):
    boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
  correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
  do_nms(boxes, 0.5)
  v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
  dets=[]
  for i in range(len(v_boxes)):
    if v_labels[i]=='car':
      box = v_boxes[i]
      y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
      dets.append([x1,y1,x2,y2,v_scores[i]])
  dets=np.array(dets)
  trackers = object_tracker.update(dets)
  tracks=[]
  for d in trackers:
    d = d.astype(np.int32)
    tracks.append(d.tolist())
  print(tracks)
  save_tracks.append(tracks)

--2021-05-31 09:51:42--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  96.8MB/s    in 2.4s    

2021-05-31 09:51:44 (96.8 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

     |████████████████████████████████| 184kB 8.6MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110452 sha256=9c13a1483b8aef21a412d1fb4d24187dfe42eaa94379d0a1d249c71cd6910e07
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy
1
[[416, 335, 497, 365, 9], [287, 295, 394, 365, 8], [782, 581, 1073, 813, 7], [1188, 481, 1505, 699, 6], [625, 450, 895, 611, 5], [578, 372, 829, 527, 4], [891, 